In [1]:
import pysam as ps
import numpy as np
import collections
import matplotlib.pyplot as plt
from numpy.core.defchararray import array, center
import pysam as ps
import numpy as np
from collections import defaultdict
from scipy.ndimage.measurements import label, standard_deviation
from scipy.stats.mstats_basic import kstest, normaltest
from sklearn.cluster import KMeans
import sys
from scipy.signal import savgol_filter
import math
from subprocess import call
import os.path
from utils import Gene, TSS, Point
from scipy import stats
from sklearn import svm
import sympy
import math
from math import e
import random
from keras import Model
from keras.models import load_model
import numpy as np
import os
import pandas as pd

2021-11-18 18:54:49.920935: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-18 18:54:49.920973: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
dic = {
    'gene':'./data/gene/GRCh37.gene.bed',
    'non_gene':'./data/gene/non_gene.bed',
    'non_gene_4':'./data/gene/non_gene_4.bed',
    'fasta':'/home/jiay/Desktop/hg19/hg19.fa',
    'bam1':'./data/051.bam',
    'bam2':'./data/051_2.bam',
    'bam3':'./data/051_3.bam',
    'bam123':'./data/051_123.bam',
    'bam4':'./data/051_4.bam',
    'bam5':'./data/051_5.bam',
    'TSS_low':'./data/gene/low_expressed.bed',
    'TSS_HK':'./data/gene/HK.bed',
    'TSS_silent':'./data/gene/silent_gene_TSS.bed',
    'ATAC_hema_1':'./data/gene/ATAC_hema_1.bed',
    'ATAC_hema_2':'./data/gene/ATAC_hema_2.bed'
    }
TSS_HK = []
# TSS_test_x = []
# with open(dic['ATAC_hema_1'],'r') as f:
#     for line in f:
#         ll = line.strip().split('\t')
#         if ll[0] in ['1']:
#             TSS_HK.append(TSS(ll[0], int((int(ll[1])+int(ll[2]))/2)))
with open(dic['TSS_HK'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] in ['1','2','3']:
            TSS_HK.append(TSS(ll[0], int(int(ll[1])+1000)))
            
        


TSS_NonGene = []
with open(dic['non_gene'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        TSS_NonGene.append(TSS(ll[0], int(ll[1])+int(1000)))
# TSS_NonGene_4 = []
# with open(dic['non_gene_4'],'r') as f:
#     for line in f:
#         ll = line.strip().split('\t')
#         TSS_NonGene_4.append(TSS(ll[0], int(ll[1])+int(1000)))


bamfile = ps.AlignmentFile(dic['bam123'],'rb')

# TSS_test_x += TSS_NonGene[len(TSS_HK):len(TSS_HK)+len(TSS_test_x)]
# perm_test = random.sample(range(len(TSS_test_x)),len(TSS_test_x))
# TSS_test_x = np.array(TSS_test_x)
# TSS_test_y = np.array(TSS_test_y)
# TSS_test_x = TSS_test_x[perm_test[:len(perm_test)]]
# TSS_test_y = TSS_test_y[perm_test[:len(perm_test)]]



TSS_NonGene = TSS_NonGene[:len(TSS_HK)]

TSSes_x = TSS_HK + TSS_NonGene
TSSes_y = [1]*len(TSS_HK) + [0]*len(TSS_NonGene)
perm = random.sample(range(len(TSSes_x)),len(TSSes_x))
TSSes_x = np.array(TSSes_x)
TSSes_y = np.array(TSSes_y)
TSSes_x = TSSes_x[perm[:len(perm)]]
TSSes_y = TSSes_y[perm[:len(perm)]]
labels = TSSes_y
up = 1000
down = 1000

In [3]:
'''lstm_input'''

feature_matrix = []
for j, tss in enumerate(TSSes_x):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    up_end = np.zeros(up+down, dtype= int)
    down_end = np.zeros(up+down, dtype= int)
    long = np.zeros(up+down, dtype= int)
    short = np.zeros(up+down, dtype= int)
    cov = np.zeros(up+down, dtype= int)
    for r in bamfile.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            # ss = max(0, r.reference_start - start)
            # ee = min(r.reference_start - start + abs(r.isize), end - start)
            ss = r.reference_start - start
            ee = r.reference_start - start + abs(r.isize)
            if ss >= 0:
                up_end[ss] += 1
            else:
                ss = 0
            if ee < end - start:
                down_end[ee] += 1
            else:
                ee = end - start
            for i in range(ss, ee):
                cov[i] += 1
            if 200 >= abs(r.isize) > 130:
                for i in range(ss, ee):
                    long[i] += 1
            if abs(r.isize) <= 130:
                for i in range(ss, ee):
                    short[i] += 1
    k = 0
    win = 40
    feature_win = np.zeros((int((up+down)/win), 4), dtype= int)
    while k < (up+down)/win:
        ss = k * win
        ee = k * win + win
        ff = []
        ff.append(int(round(np.mean(cov[ss:ee]))))
        ff.append(int(round(np.mean(long[ss:ee]-short[ss:ee]))))
        ff.append(int(round(np.sum(abs(up_end[ss:ee]-down_end[ss:ee])))))
        wps = 0
        for r in bamfile.fetch(chrom[-1], ss+start, ee+start):
            if r.reference_start <= ss+start and r.reference_start+abs(r.isize) >= ee + start:
                wps += 1
            elif ee + start > r.reference_start > ss + start:
                wps -= 1
            elif ss+start < r.reference_start + abs(r.isize) < ee + start:
                wps -= 1
        ff.append(int(wps))
        feature_win[k] = np.array(ff)
        k =  k + 1
    feature_matrix.append(feature_win)
feature_matrix = np.array(feature_matrix)

lstm_x = []
for mat in feature_matrix:
    lstm_x.append(mat)
lstm_x = np.array(lstm_x)


In [4]:
'''cnn_input'''
raw_data = []
for j, tss in enumerate(TSSes_x):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
#    distribution_matrix = np.zeros((int(up+down), 200), dtype=int)
    distribution_matrix = np.zeros((200,200),dtype=int)
    for r in bamfile.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse and 50 < abs(r.isize) < 250:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            if r.reference_start < start:
                continue
            if r.reference_start + abs(r.isize) > end:
                continue
            ss = max(0, r.reference_start - start)
            relative_isize = abs(r.isize)-50
            distribution_matrix[ss//10,relative_isize] += 1
    raw_data.append(distribution_matrix)
raw_data = np.array(raw_data)

cnn_x = []
for mat in raw_data:
    cnn_x.append(mat)
cnn_x = np.array(cnn_x)

In [5]:
train_cnn_x = cnn_x[:int(len(cnn_x)*0.8)]
train_lstm_x = lstm_x[:int(len(lstm_x)*0.8)]
train_y = labels[:int(len(labels)*0.8)]
test_cnn_x = cnn_x[int(len(cnn_x)*0.8):]
test_lstm_x = lstm_x[int(len(lstm_x)*0.8):]
test_y = labels[int(len(labels)*0.8):]

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *
LL = keras.utils.to_categorical(labels)
def attention_3d_block(inputs):
    a = Permute((2, 1))(inputs)
    a = Dense(50, activation='softmax')(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = layers.multiply([inputs, a_probs], name='attention_mul')
    return output_attention_mul
def get_attention_model(cnn_x, lstm_x):
    cnn_input = Input(shape=(cnn_x.shape[1],cnn_x.shape[2]), name='cnn_input')
    conv1 = Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same')(cnn_input)
    cnn_output = AveragePooling1D(pool_size=2, strides=2)(conv1)
    flatten = Flatten()(cnn_output)
    aux_output = Dense(2, activation='softmax', name='aux_output')(flatten)
    aux_input = Input(shape=(lstm_x.shape[1], lstm_x.shape[2]), name='aux_input')
    x = concatenate([cnn_output, aux_input])
    lstm_out = Bidirectional(LSTM(50, return_sequences=True),name='0')(x)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)
    main_output = Dense(2, activation='softmax', kernel_regularizer=None, bias_regularizer=None,name='6')(attention_mul)
    model = Model(inputs=[cnn_input, aux_input], outputs=[main_output, aux_output])
    return model
model = get_attention_model(cnn_x,lstm_x)
# model.summary()
adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit([cnn_x, lstm_x], [LL, LL], epochs=300, batch_size=128, validation_split=0.1,callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./conv_lstm.h5', save_best_only=True)],
                        shuffle=True, verbose=2)

Epoch 1/300
12/12 - 6s - loss: 1.3865 - 6_loss: 0.6929 - aux_output_loss: 0.6936 - 6_accuracy: 0.5271 - aux_output_accuracy: 0.5095 - val_loss: 1.3811 - val_6_loss: 0.6916 - val_aux_output_loss: 0.6895 - val_6_accuracy: 0.6463 - val_aux_output_accuracy: 0.5488
Epoch 2/300


/home/jiay/.local/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


12/12 - 2s - loss: 1.3797 - 6_loss: 0.6905 - aux_output_loss: 0.6892 - 6_accuracy: 0.7809 - aux_output_accuracy: 0.5380 - val_loss: 1.3742 - val_6_loss: 0.6890 - val_aux_output_loss: 0.6851 - val_6_accuracy: 0.8963 - val_aux_output_accuracy: 0.5732
Epoch 3/300
12/12 - 1s - loss: 1.3738 - 6_loss: 0.6879 - aux_output_loss: 0.6859 - 6_accuracy: 0.8908 - aux_output_accuracy: 0.5305 - val_loss: 1.3677 - val_6_loss: 0.6861 - val_aux_output_loss: 0.6815 - val_6_accuracy: 0.9024 - val_aux_output_accuracy: 0.5671
Epoch 4/300
12/12 - 1s - loss: 1.3679 - 6_loss: 0.6849 - aux_output_loss: 0.6830 - 6_accuracy: 0.9009 - aux_output_accuracy: 0.5217 - val_loss: 1.3618 - val_6_loss: 0.6828 - val_aux_output_loss: 0.6789 - val_6_accuracy: 0.9268 - val_aux_output_accuracy: 0.5427
Epoch 5/300
12/12 - 1s - loss: 1.3620 - 6_loss: 0.6814 - aux_output_loss: 0.6806 - 6_accuracy: 0.9030 - aux_output_accuracy: 0.5115 - val_loss: 1.3550 - val_6_loss: 0.6786 - val_aux_output_loss: 0.6764 - val_6_accuracy: 0.9329 - 

In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *
def attention_3d_block(inputs):
    a = Permute((2, 1))(inputs)
    a = Dense(50, activation='softmax')(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = layers.multiply([inputs, a_probs], name='attention_mul')
    return output_attention_mul
def get_attention_model(cnn_x, lstm_x):
    cnn_input = Input(shape=(cnn_x.shape[1],cnn_x.shape[2]), name='cnn_input')
    conv1 = Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same')(cnn_input)
    cnn_output = AveragePooling1D(pool_size=2, strides=2)(conv1)
    flatten = Flatten()(cnn_output)
    aux_output = Dense(1, activation='sigmoid', name='aux_output')(flatten)
    aux_input = Input(shape=(lstm_x.shape[1], lstm_x.shape[2]), name='aux_input')
    x = concatenate([cnn_output, aux_input])
    lstm_out = Bidirectional(LSTM(50, return_sequences=True),name='0')(x)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)
    main_output = Dense(1, activation='sigmoid', kernel_regularizer=None, bias_regularizer=None,name='6')(attention_mul)
    model = Model(inputs=[cnn_input, aux_input], outputs=[main_output, aux_output])
    return model
model = get_attention_model(cnn_x,lstm_x)
# model.summary()
adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit([cnn_x, lstm_x], [labels, labels], epochs=300, batch_size=128, validation_split=0.1,callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./conv_lstm.h5', save_best_only=True)],
                        shuffle=True, verbose=2)


Epoch 1/300
12/12 - 6s - loss: 1.3814 - 6_loss: 0.6902 - aux_output_loss: 0.6912 - 6_accuracy: 0.4973 - aux_output_accuracy: 0.5054 - val_loss: 1.3716 - val_6_loss: 0.6869 - val_aux_output_loss: 0.6847 - val_6_accuracy: 0.5305 - val_aux_output_accuracy: 0.5610
Epoch 2/300
12/12 - 1s - loss: 1.3744 - 6_loss: 0.6862 - aux_output_loss: 0.6882 - 6_accuracy: 0.5244 - aux_output_accuracy: 0.5047 - val_loss: 1.3653 - val_6_loss: 0.6826 - val_aux_output_loss: 0.6827 - val_6_accuracy: 0.5854 - val_aux_output_accuracy: 0.5610
Epoch 3/300
12/12 - 1s - loss: 1.3676 - 6_loss: 0.6818 - aux_output_loss: 0.6858 - 6_accuracy: 0.6147 - aux_output_accuracy: 0.5068 - val_loss: 1.3585 - val_6_loss: 0.6778 - val_aux_output_loss: 0.6807 - val_6_accuracy: 0.6402 - val_aux_output_accuracy: 0.5488
Epoch 4/300
12/12 - 1s - loss: 1.3604 - 6_loss: 0.6769 - aux_output_loss: 0.6834 - 6_accuracy: 0.7083 - aux_output_accuracy: 0.5068 - val_loss: 1.3512 - val_6_loss: 0.6723 - val_aux_output_loss: 0.6789 - val_6_accurac

In [17]:
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, Convolution1D, MaxPooling1D, Flatten, Bidirectional,Dropout, AveragePooling1D
from keras.models import Model
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

cnn_input = Input(shape=(cnn_x.shape[1],cnn_x.shape[2]), name='cnn_input')
conv1 = Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same')(cnn_input)
cnn_output = AveragePooling1D(pool_size=2, strides=2)(conv1)
# conv2 = Convolution1D(filters=128, kernel_size=2, strides=2, activation='relu', padding='same')(pooling1)
# cnn_output = MaxPooling1D(pool_size=2, strides=2)(conv2)
flatten = Flatten()(cnn_output)
aux_output = Dense(1, activation='sigmoid', name='aux_output')(flatten)

aux_input = Input(shape=(lstm_x.shape[1], lstm_x.shape[2]), name='aux_input')
x = concatenate([cnn_output, aux_input])

lstm_out = Bidirectional(LSTM(50, return_sequences=True),name='0')(x)
conv = Convolution1D(filters=100, kernel_size=3, activation='relu',strides=1,padding='same',name='1')(lstm_out)
pool = MaxPooling1D(pool_size=2, strides=2,name='2')(conv)
drop = Dropout(0.2)(pool)
flatten = Flatten()(drop)
dense = Dense(300, activation='relu', kernel_regularizer=None, bias_regularizer=None,name='4')(flatten)
drop = Dropout(0.2)(dense)
main_output = Dense(1, activation='sigmoid', kernel_regularizer=None, bias_regularizer=None,name='6')(drop)

model = Model(inputs=[cnn_input, aux_input], outputs=[main_output, aux_output])

# print(model.summary())
adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit([cnn_x, lstm_x], [labels, labels], epochs=300, batch_size=128, validation_split=0.1,callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./conv_lstm.h5', save_best_only=True)],
                        shuffle=True, verbose=2)

Epoch 1/300
12/12 - 5s - loss: 1.3355 - 6_loss: 0.6334 - aux_output_loss: 0.7021 - 6_accuracy: 0.6737 - aux_output_accuracy: 0.4600 - val_loss: 1.1889 - val_6_loss: 0.4917 - val_aux_output_loss: 0.6973 - val_6_accuracy: 0.9390 - val_aux_output_accuracy: 0.4512
Epoch 2/300
12/12 - 1s - loss: 1.1316 - 6_loss: 0.4356 - aux_output_loss: 0.6960 - 6_accuracy: 0.8867 - aux_output_accuracy: 0.4878 - val_loss: 1.0189 - val_6_loss: 0.3274 - val_aux_output_loss: 0.6914 - val_6_accuracy: 0.9329 - val_aux_output_accuracy: 0.4878
Epoch 3/300
12/12 - 1s - loss: 0.9799 - 6_loss: 0.2883 - aux_output_loss: 0.6916 - 6_accuracy: 0.9186 - aux_output_accuracy: 0.5020 - val_loss: 0.9331 - val_6_loss: 0.2459 - val_aux_output_loss: 0.6872 - val_6_accuracy: 0.9329 - val_aux_output_accuracy: 0.5122
Epoch 4/300
12/12 - 1s - loss: 0.9313 - 6_loss: 0.2433 - aux_output_loss: 0.6881 - 6_accuracy: 0.9213 - aux_output_accuracy: 0.5081 - val_loss: 0.9175 - val_6_loss: 0.2334 - val_aux_output_loss: 0.6841 - val_6_accurac

In [10]:
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, Convolution1D, MaxPooling1D, Flatten, Bidirectional,Dropout, AveragePooling1D
from keras.models import Model
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

LL = keras.utils.to_categorical(labels)

cnn_input = Input(shape=(cnn_x.shape[1],cnn_x.shape[2]), name='cnn_input')
conv1 = Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same')(cnn_input)
cnn_output = AveragePooling1D(pool_size=2, strides=2)(conv1)
# conv2 = Convolution1D(filters=128, kernel_size=2, strides=2, activation='relu', padding='same')(pooling1)
# cnn_output = MaxPooling1D(pool_size=2, strides=2)(conv2)
flatten = Flatten()(cnn_output)
aux_output = Dense(2, activation='softmax', name='aux_output')(flatten)

aux_input = Input(shape=(lstm_x.shape[1], lstm_x.shape[2]), name='aux_input')
x = concatenate([cnn_output, aux_input])

lstm_out = Bidirectional(LSTM(50, return_sequences=True),name='0')(x)
conv = Convolution1D(filters=100, kernel_size=3, activation='relu',strides=1,padding='same',name='1')(lstm_out)
pool = MaxPooling1D(pool_size=2, strides=2,name='2')(conv)
drop = Dropout(0.2)(pool)
flatten = Flatten()(drop)
dense = Dense(300, activation='relu', kernel_regularizer=None, bias_regularizer=None,name='4')(flatten)
drop = Dropout(0.2)(dense)
main_output = Dense(2, activation='softmax', kernel_regularizer=None, bias_regularizer=None,name='6')(drop)

model = Model(inputs=[cnn_input, aux_input], outputs=[main_output, aux_output])

# print(model.summary())
adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit([cnn_x, lstm_x], [LL, LL], epochs=50, batch_size=128, validation_split=0.1,callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./conv_lstm.h5', save_best_only=True)],
                        shuffle=True, verbose=2)

/home/jiay/.local/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/50
12/12 - 4s - loss: 1.1902 - 6_loss: 0.5005 - aux_output_loss: 0.6896 - 6_accuracy: 0.7795 - aux_output_accuracy: 0.5000 - val_loss: 0.9836 - val_6_loss: 0.2994 - val_aux_output_loss: 0.6843 - val_6_accuracy: 0.9390 - val_aux_output_accuracy: 0.5183
Epoch 2/50
12/12 - 1s - loss: 0.9554 - 6_loss: 0.2704 - aux_output_loss: 0.6850 - 6_accuracy: 0.9111 - aux_output_accuracy: 0.5041 - val_loss: 0.9082 - val_6_loss: 0.2277 - val_aux_output_loss: 0.6806 - val_6_accuracy: 0.9512 - val_aux_output_accuracy: 0.5183
Epoch 3/50
12/12 - 1s - loss: 0.9080 - 6_loss: 0.2273 - aux_output_loss: 0.6807 - 6_accuracy: 0.9193 - aux_output_accuracy: 0.5061 - val_loss: 0.8946 - val_6_loss: 0.2170 - val_aux_output_loss: 0.6776 - val_6_accuracy: 0.9390 - val_aux_output_accuracy: 0.5183
Epoch 4/50
12/12 - 1s - loss: 0.8995 - 6_loss: 0.2228 - aux_output_loss: 0.6767 - 6_accuracy: 0.9172 - aux_output_accuracy: 0.5095 - val_loss: 0.8889 - val_6_loss: 0.2143 - val_aux_output_loss: 0.6745 - val_6_accuracy: 0

In [28]:
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, Convolution1D, MaxPooling1D, Flatten, Bidirectional,Dropout, AveragePooling1D
from keras.models import Model
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

cnn_input = Input(shape=(cnn_x.shape[1], cnn_x.shape[2]), name='cnn_input')
lstm_input = Input(shape=(lstm_x.shape[1], lstm_x.shape[2]), name='lstm_input')
conv1 = Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same')(cnn_input)
pool1 = AveragePooling1D(pool_size=2, strides=2)(conv1)
cnn_output = Model(inputs=cnn_input, outputs=pool1)
main_input = concatenate([cnn_output.output, lstm_input])

lstm_out = Bidirectional(LSTM(50, return_sequences=True),name='0')(main_input)
conv = Convolution1D(filters=100, kernel_size=3, activation='relu',strides=1,padding='same',name='1')(lstm_out)
pool = MaxPooling1D(pool_size=2, strides=2,name='2')(conv)
drop = Dropout(0.2)(pool)
flatten = Flatten()(drop)
dense = Dense(300, activation='relu', kernel_regularizer=None, bias_regularizer=None,name='4')(flatten)
drop = Dropout(0.2)(dense)
main_output = Dense(1, activation='sigmoid', kernel_regularizer=None, bias_regularizer=None,name='6')(drop)

model = Model(inputs=[cnn_output.input, lstm_input], outputs=main_output)


adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit([cnn_x, lstm_x], labels, epochs=300, batch_size=128, validation_split=0.1,callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./conv_lstm.h5', save_best_only=True)],
                        shuffle=True, verbose=2)






/home/jiay/.local/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cnn_input (InputLayer)          [(None, 200, 200)]   0                                            
__________________________________________________________________________________________________
conv1d_18 (Conv1D)              (None, 100, 32)      12832       cnn_input[0][0]                  
__________________________________________________________________________________________________
average_pooling1d_13 (AveragePo (None, 50, 32)       0           conv1d_18[0][0]                  
__________________________________________________________________________________________________
lstm_input (InputLayer)         [(None, 50, 4)]      0                                            
___________________________________________________________________________________________

In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import random
# cnn_x = cnn_x.reshape(cnn_x.shape[0], cnn_x.shape[1], cnn_x.shape[2], 1)
time_steps = cnn_x.shape[1]
input_size = cnn_x.shape[2]
model = keras.models.Sequential()

# model.add(layers.Conv2D(filters=32,kernel_size=(2,2),strides=(1,1),padding='same',input_shape=(time_steps, input_size,1),name='0'))
# model.add(layers.Activation('relu'))
# model.add(layers.MaxPool2D(strides=(2,2),pool_size=(2,2),name='1'))
# model.add(layers.Conv2D(filters=64,kernel_size=3,strides=(1,1),padding='same',name='2'))
# model.add(layers.Activation('relu'))
# model.add(layers.MaxPool2D(strides=(2,2),pool_size=(2,2),name='3'))

model.add(layers.Conv1D(filters=32, kernel_size=2, strides=2, padding='same', input_shape=(time_steps,input_size)))
model.add(layers.Activation('relu'))
model.add(layers.AveragePooling1D(strides=2, pool_size=2))
# model.add(layers.Conv1D(filters=64, kernel_size=2,strides=2, padding='same'))
# model.add(layers.Activation('relu'))
# model.add(layers.AveragePooling1D(strides=2, pool_size=2))

model.add(layers.Flatten())
model.add(layers.Dense(750,activation='relu', kernel_regularizer=None, bias_regularizer=None,name='4'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(300,activation='relu',kernel_regularizer=None, bias_regularizer=None,name='5'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(16,activation='relu',kernel_regularizer=None, bias_regularizer=None,name='6'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation='sigmoid',kernel_regularizer=None, bias_regularizer=None,name='7'))
adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(cnn_x, TSSes_y, epochs=50, batch_size=128, validation_split=0.1)

Epoch 1/300


ValueError: in user code:

    /home/jiay/.local/lib/python3.8/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /home/jiay/.local/lib/python3.8/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/jiay/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/jiay/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/jiay/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/jiay/.local/lib/python3.8/site-packages/keras/engine/training.py:835 run_step  **
        outputs = model.train_step(da

In [23]:
# '''cnn_input'''
# raw_data = []
# for j, tss in enumerate(TSSes_x):
#     chrom = tss.chrom
#     start = tss.pos - up
#     end = tss.pos + down
# #    distribution_matrix = np.zeros((int(up+down), 200), dtype=int)
#     distribution_matrix = np.zeros((200,200),dtype=int)
#     for r in bamfile.fetch(chrom[-1], start-500, end + 500):
#         if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse and 50 < abs(r.isize) < 250:
#             if r.reference_start + abs(r.isize) < start:
#                 continue
#             if r.reference_start >= end:
#                 continue
#             if r.reference_start < start:
#                 continue
#             if r.reference_start + abs(r.isize) > end:
#                 continue
#             ss = max(0, r.reference_start - start)
#             relative_isize = abs(r.isize)-50
#             distribution_matrix[ss//10,relative_isize] += 1
#     raw_data.append(distribution_matrix)
# raw_data = np.array(raw_data)

# cnn_x = []
# for mat in raw_data:
#     cnn_x.append(mat)
# cnn_x = np.array(cnn_x)


In [24]:
print(cnn_x.shape)
print(lstm_x.shape)

(1638, 40, 40)
(1638, 50, 4)


Epoch 1/50
12/12 [==============================] - 4s 125ms/step - loss: 1.2463 - 6_loss: 0.5551 - aux_output_loss: 0.6912 - 6_accuracy: 0.7605 - aux_output_accuracy: 0.5095 - val_loss: 1.0885 - val_6_loss: 0.3941 - val_aux_output_loss: 0.6944 - val_6_accuracy: 0.8963 - val_aux_output_accuracy: 0.5061
Epoch 2/50
12/12 [==============================] - 1s 80ms/step - loss: 1.0094 - 6_loss: 0.3233 - aux_output_loss: 0.6861 - 6_accuracy: 0.8786 - aux_output_accuracy: 0.5299 - val_loss: 0.9998 - val_6_loss: 0.3074 - val_aux_output_loss: 0.6924 - val_6_accuracy: 0.8963 - val_aux_output_accuracy: 0.5000
Epoch 3/50
12/12 [==============================] - 1s 88ms/step - loss: 0.9389 - 6_loss: 0.2569 - aux_output_loss: 0.6819 - 6_accuracy: 0.9016 - aux_output_accuracy: 0.5224 - val_loss: 0.9800 - val_6_loss: 0.2896 - val_aux_output_loss: 0.6904 - val_6_accuracy: 0.9146 - val_aux_output_accuracy: 0.5061
Epoch 4/50
12/12 [==============================] - 1s 91ms/step - loss: 0.9229 - 6_loss: 

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import random
# cnn_x = cnn_x.reshape(cnn_x.shape[0], cnn_x.shape[1], cnn_x.shape[2], 1)
time_steps = cnn_x.shape[1]
input_size = cnn_x.shape[2]
model = keras.models.Sequential()

model.add(keras.layers.Convolution1D(filters=128,kernel_size=2,input_shape=(time_steps,input_size),activation='relu',strides=2,padding='same'))
model.add(keras.layers.MaxPooling1D(pool_size=2,strides=2))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Convolution1D(filters=128,kernel_size=2,input_shape=(time_steps,input_size),activation='relu',strides=2,padding='same'))
model.add(keras.layers.MaxPooling1D(pool_size=2,strides=2))
model.add(keras.layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid',kernel_regularizer=None, bias_regularizer=None,name='7'))
adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(cnn_x, TSSes_y, epochs=50, batch_size=128, validation_split=0.1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 400, 128)          51328     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 200, 128)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 200, 128)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 100, 128)          32896     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 50, 128)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 50, 128)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6400)             

2021-11-11 16:37:14.990916: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1886720000 exceeds 10% of free system memory.


Epoch 1/50
12/12 [==============================] - 5s 268ms/step - loss: 0.6862 - accuracy: 0.5020 - val_loss: 0.6795 - val_accuracy: 0.5244
Epoch 2/50
12/12 [==============================] - 2s 156ms/step - loss: 0.6790 - accuracy: 0.5020 - val_loss: 0.6738 - val_accuracy: 0.5244
Epoch 3/50
12/12 [==============================] - 2s 170ms/step - loss: 0.6725 - accuracy: 0.5332 - val_loss: 0.6692 - val_accuracy: 0.5488
Epoch 4/50
12/12 [==============================] - 2s 174ms/step - loss: 0.6645 - accuracy: 0.5753 - val_loss: 0.6634 - val_accuracy: 0.5854
Epoch 5/50
12/12 [==============================] - 2s 172ms/step - loss: 0.6549 - accuracy: 0.6092 - val_loss: 0.6569 - val_accuracy: 0.6037
Epoch 6/50
12/12 [==============================] - 2s 165ms/step - loss: 0.6448 - accuracy: 0.6995 - val_loss: 0.6506 - val_accuracy: 0.6951
Epoch 7/50
12/12 [==============================] - 2s 169ms/step - loss: 0.6368 - accuracy: 0.7171 - val_loss: 0.6413 - val_accuracy: 0.6890
Epoch 

KeyboardInterrupt: 